https://python.langchain.com/docs/how_to/custom_tools/

In [1]:
from getpass import getpass

In [2]:
OPENAI_API_KEY = getpass(prompt='Enter your OpenAI Token:')

Enter your OpenAI Token: ········


In [3]:
from ase.build import bulk

In [4]:
from ase.data import reference_states, atomic_numbers

In [5]:
from langchain.agents import tool

In [6]:
@tool
def get_crystal_structure(chemical_symbol: str) -> str:
    """Returns the atomic crystal structure of a chemcial symbol"""
    ref_state = reference_states[atomic_numbers[chemical_symbol]]
    if ref_state is None:
        return "No crystal structure known."
    else:
        return ref_state["symmetry"]

In [7]:
get_crystal_structure.invoke("Fe")

'bcc'

In [8]:
get_crystal_structure.invoke("Au")

'fcc'

In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)

In [10]:
tools = [get_crystal_structure]

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            # "You are very powerful assistant, but don't know current events.",  # This initial query fails when car is provided as input rather than gold
            "You are very powerful assistant, but don't know current events. For each query vailidate that it contains a chemical element and otherwise cancel.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [12]:
llm_with_tools = llm.bind_tools(tools)

In [13]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [14]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [15]:
lst = list(agent_executor.stream({"input": "What is the crystal structure of gold"}))  # Yeah this worked !!



> Entering new AgentExecutor chain...

Invoking: `get_crystal_structure` with `{'chemical_symbol': 'Au'}`


fccThe crystal structure of gold is face-centered cubic (fcc).

> Finished chain.


In [16]:
lst = list(agent_executor.stream({"input": "What is the crystal structure of car"}))  # I did not know cars were made of carbon



> Entering new AgentExecutor chain...
I'm sorry, but the query does not contain a chemical element. Please provide a valid chemical element for me to determine its crystal structure.

> Finished chain.


# Lessons learned: 
* It works surprisingly well
* Modifying the system prompt helps us to validate that the user provides a chemical element as part of the input. 